# Algorithms on linked lists

We use the `List` class defined as follows. For convenience we implement the `__str__` method which lets us use the built-in `print` function.
```python
example_list = List(24)
print(example_list)
```
```
>>> ~[ 24 ]~
```

In [ ]:
class List:
    def __init__(self, data):
        '''
        Linked list containing input data.
        '''
        self.data = data
        self.next = None
        
    def __str__(self):
        '''
        String representation of a linked list.
        '''
        values = '~['
        l = self
        while l is not None:
            values +=  f' {l.data} '
            l = l.next
        values += ']~'
        return values

In [ ]:
my_list = List(1)
my_list.next = List(6)
my_list.next.next = List(3)

In [ ]:
print(my_list)

In [ ]:
def as_linked_list(container):
    '''
    Create a linked list representation of a given container.
    
    Parameters
    ----------
    container: any object supporting the 'for ... in ...' syntax.
    
    Returns
    -------
    List
        a 'List' of the contained elements.
    '''
    head = List(None)
    result = head
    for i in container:
        result.next = List(i)
        result = result.next
    return head.next

In [ ]:
print(as_linked_list(range(5)))

## Compute the sum of even numbers from a list

We start by defining our version of `filter`.

In [ ]:
def filtered_by(condition, l):
    '''
    List all elements that fulfill the condition.
    
    Parameters
    ----------
    condition: function of type (List -> bool).
    
    l: List to be filtered
    
    Returns
    -------
    List
        containing the elements for which the condition is True.
    
    Notes
    -----
    If no element fulfills the condition, the returned list will be empty.
    '''
    head = List(None)
    result = head
    while l is not None:
        if condition(l):
            result.next = List(l.data)
            result = result.next
        l = l.next
    return head.next

In [ ]:
print(my_list)
print(filtered_by(lambda x: x.data % 2 == 1, my_list))

We also define a version of `itertools.reduce` that works on `List`s.

In [ ]:
def reduce_with(operation, l, init):
    '''
    Accumulate elements from a list with an operation starting from a given value.
    
    Parameters
    ----------
    operation: function of type (t, t -> t)
    
    l: List of elements of type t
    
    init: value fo type t
    
    Returns
    -------
    value of type t
        given a list of elements [a1, a2, ..., aN] and
        writing x * y for operation(x, y)
        computes the following expression:
        (...((init * a1) * a2) ... * aN)
    '''
    result = init
    while l is not None:
        result = operation(result, l.data)
        l = l.next
    return result

The sum and the product of elements in a list are easy to define with `reduce_with`.

In [ ]:
def summed(l):
    '''
    Sum elements of a list of numbers.
    
    Parameters
    ----------
    l: List of elements that support 'lambda x: x + 0'.
    
    Returns
    -------
    value of one of the number types
        the sum.
    
    Notes
    -----
    - the neutral element for addition is 0.
    - if values are of different number types, the most general will be used.
    '''
    return reduce_with(lambda a, b: a + b, l, 0)

def multiplied(l):
    '''
    Multiply elements of a list of numbers.
    
    Parameters
    ----------
    l: List of elements that support 'lambda x: x * 1'.
    
    Returns
    -------
    value of one of the number types
        the product.
    
    Notes
    -----
    - the neutral element for multiplication is 1.
    - if values are of different number types, the most general type will be used.
    '''
    return reduce_with(lambda a, b: a * b, l, 1)

In [ ]:
print(my_list)
print('sum:', summed(my_list), 'product:', multiplied(my_list))

We can now get the sum of even numbers by summing over a filtered list.

In [ ]:
def sum_even_numbers(l):
    '''
    Sum the even integers of a list of numbers.
    
    Parameters
    ----------
    l: List of elements that support 'lambda x: x % 2'.
    
    Returns
    -------
    value of one of the number types
        the sum of even integers.
    '''
    return summed(filtered_by(lambda x: x.data % 2 == 0, l))

In [ ]:
import random

random_list = as_linked_list(random.sample(range(10, 500, 5), k=10))
print(random_list)
print(filtered_by(lambda x: x.data % 2 == 0, random_list))
print('sum:', sum_even_numbers(random_list))

## Return a list with every other element

Instead of doing it the direct way, we start by creating our own versions of `map` and `enumerate`.

In [ ]:
def mapped_with(function, l):
    '''
    Apply function to each element of the list.
    
    Parameters
    ----------
    function: any function (preferably pure) of type (a -> b).
    
    l: a List of elements of type a.
    
    Returns
    -------
    a List of elements of type b.
    
    Notes
    -----
    Can be used with a function of type (a -> a) since
    (a -> a) is an instance of (a -> b).
    '''
    head_result = List(None)
    head = head_result
    while l is not None:
        head.next = List(function(l.data))
        l, head = l.next, head.next
    return head_result.next

In [ ]:
print(mapped_with(lambda x: x**2, random_list))

We need to be able to `zip` lists to facilitate enumeration.

In [ ]:
def zipped(left, right):
    '''
    Combine two lists into a list of pairs.
    
    Parameters
    ----------
    left: List.
    
    right: List.
    
    Returns
    -------
    List
        the data of each element of the list is a pair of data obtained
        from iterating through both list at the same time.
        
    Notes
    -----
    The length of the list is equal to the length of the shortest list given as input.
    '''
    head = List(None)
    result = head
    while left is not None and right is not None:
        result.next = List((left.data, right.data))
        result, left, right = result.next, left.next, right.next
    return head.next

Indeed enumeration means to associate a range of indices (here from 0 to the length of the list - 1 to follow Pythonic conventions) with a list

In [ ]:
def length(l):
    result = 0
    while l is not None:
        result += 1
        l = l.next
    return result

def enumerated(l):
    return zipped(as_linked_list(range(length(l))), l)

In [ ]:
print(zipped(random_list, mapped_with(lambda x: x**2, random_list)))

In [ ]:
print(as_linked_list(range(6)))

In [ ]:
my_enumerated_list = enumerated(my_list)
print(my_enumerated_list)

In [ ]:
print(filtered_by(lambda x: x.data[0] % 2 == 0, my_enumerated_list))

We need to retrieve an enumerated list but it turns out that this is simply unzipping the lists.

In [ ]:
def unzip(l):
    '''
    Extract pairs of values from a list into two lists.
    
    Parameters
    ----------
    l: List of pairs.
    
    Returns
    -------
    (List, List)
        the left list contains the left elements of the pairs while
        the right list contains the right elements.
    '''
    head_left, head_right = List(None), List(None)
    left, right = head_left, head_right
    while l is not None:
        left.next, right.next = List(l.data[0]), List(l.data[1])
        l, left, right = l.next, left.next, right.next
    return head_left.next, head_right.next

In [ ]:
print(random_list)
left, right = unzip(
    filtered_by(lambda x: x.data[0] % 2 == 0, enumerated(random_list)))
print(right)

We can check that our implementations of `zip` and `unzip` (which is not implemented by default in Python) follow the law:
```python
a, b == unzip(zip(a, b))
```

In [ ]:
def equals(left, right):
    '''
    Test if two lists contain the same elements.
    
    Parameters
    ----------
    left: List.
    
    right: List.
    
    Returns
    -------
    bool
        True only if both lists have the same length and contain the same elements
        
    Notes
    -----
    Assumes that lists contain values for which it makes sense
    to test difference with '!='.
    '''
    while left is not None and right is not None:
        if left.data != right.data:
            return False
        left, right = left.next, right.next
    return True if left is None and right is None else False

In [ ]:
new_left, new_right = unzip(zipped(left, right))
print(equals(left, new_left), equals(right, new_right))

In [ ]:
head = List(1)
head.next = List(6)
head.next.next = List(3)

## In-place operations

Describe how they differ from the read-only version that creates a list as output.

In [ ]:
def inplace_filter(condition, l):
    '''
    Removes elements that do not fulfill a condition.
    
    Parameters
    ----------
    condition: function of type (a -> bool)
    
    l: List of elements of type a
    
    Notes
    -------
    Modifies the input list in-place which means that it might shorten it.
    '''
    while l is not None and l.next is not None:
        if not condition(l):
            l.next = l.next.next
        l = l.next

In [ ]:
print(inplace_filter(lambda x: x.data % 2 == 1, as_linked_list(range(8))))

In [ ]:
a = as_linked_list(range(7))
print(a)
inplace_filter(lambda x: x.data % 2 == 0, a)
print(a)

In [ ]:
summed(a)

## Inplace squashing

Here we need more assumptions on our input for the computation to make sense.

In [ ]:
def inplace_squash_with(operation, list_of_even_length):
    '''
    Combine successive pairs of elements with an operation.
    
    Parameters
    ----------
    operation: function of type ((a, a) -> b).
    
    list_of_even_length: List of elements of type a
        assumed to be of even length in order to combine every pair.
    
    Returns
    -------
    List of elements of type b
        modifies the input list in-place and halves its length.
    '''
    l = list_of_even_length
    while l is not None and l.next is not None:
        l.data = operation(l.data, l.next.data)
        l.next = l.next.next
        l = l.next

def inplace_squash_sum(l):
    return inplace_squash_with(lambda a, b: a + b, l)

In [ ]:
test_range = as_linked_list(range(1,8))
print(test_range)
inplace_squash_with(lambda a, b: a + b, test_range)
print(test_range)

In [ ]:
test_range = as_linked_list(range(2,10))
print(test_range)
inplace_squash_with(lambda a, b: f'eval({a}+{b})', test_range)
print(test_range)

## Representing a set by a linked list without duplicates

In [ ]:
def concatenated(left, right):
    '''
    Create a list obtained by extending one with another.
    
    Parameters
    ----------
    left: List.
    
    right: List.
    
    Returns
    -------
    List
        concatenation of left and right.
    '''
    head = List(None)
    result = head
    while left is not None:
        result.next = List(left.data)
        result, left = result.next, left.next
    while right is not None:
        result.next = List(right.data)
        result, right = result.next, right.next
    return head.next

def belongs_to(l, elem):
    '''
    Test membership to a list.
    
    Parameters
    ----------
    l: List.
    
    elem: value of type a
        content that could be found in l.
        
    Returns
    -------
    bool
        True if an element of the list is equal to elem.
        
    Notes
    -----
    Does not support multiple indirections.
    '''
    while l is not None:
        if l.data == elem:
            return True
        l = l.next
    return False

def without_duplicates(l):
    '''
    Create a list from another without duplicate elements.
    
    Parameters
    ----------
    l: List.
    
    Returns
    -------
    List
        the new list does not contain duplicate elements and so
        is assured to be non-empty if the input list is non-empty.
    '''
    head = List(None)
    result = head
    while l is not None:
        if not belongs_to(head, l.data):
            result.next = List(l.data)
            result = result.next
        l = l.next
    return head.next

def set_union(l1, l2):
    return with_duplicates(concatenated(l1, l2))

In [ ]:
a = as_linked_list(range(1,7))
b = as_linked_list(range(3, 12, 2))
print(a, b)
c = concatenated(a, b)
print(a, b, c)

In [ ]:
belongs_to(c, 18)

In [ ]:
print(without_duplicates(c))

In [ ]:
print(without_duplicates(
    concatenated(as_linked_list([5,1,2,3]),
                as_linked_list([6,2,3])
    )))

In [ ]:
print(filtered_by(lambda x: x.data >= 6, as_linked_list([5,6,7,1,8,2,1])))

In [ ]:
class Directory:
    def __init__(self, uid, zipcode, firstname, lastname):
        '''
        directory represented as a linked list containing:
        - a unique identifier
        - zipcode information
        - first and last names.
        '''
        self.uid = uid
        self.zipcode = zipcode
        self.firstname = firstname
        self.lastname = lastname
        self.next = None
    
    def __str__(self):
        '''
        String representation of a directory.
        '''
        values = '%[\n'
        l = self
        while l is not None:
            values += f' #{l.uid}: {l.firstname} {l.lastname} | zipcode: {l.zipcode} \n'
            l = l.next
        values += '%]'
        return values

In [ ]:
me = Directory(1919, 75, 'Paul', 'Beaujean')
tristan = Directory(5654, 75, 'Tristan', 'Cazenave')
hawking = Directory(1111, 3, 'Stephen', 'Hawking')
me.next = tristan
tristan.next = hawking
directory = me

print(directory)

In [ ]:
def directory_filtered_by(condition, l):
    '''
    Directory of all entries that fulfill the condition.
    
    Parameters
    ----------
    condition: function of type (List -> bool).
    
    l: Dictionary to be filtered
    
    Returns
    -------
    Dictionary
        containing the elements for which the condition is True.
    
    Notes
    -----
    If no element fulfills the condition, the returned directory will be empty.
    '''
    head = Directory(None, None, None, None)
    result = head
    while l is not None:
        if condition(l):
            result.next = Directory(l.uid, l.zipcode, l.firstname, l.lastname)
            result = result.next
        l = l.next
    return head.next

print(directory_filtered_by(lambda x: x.zipcode == 75, directory))

In [ ]:
def minimum(before, l):
    '''
    Find the cell of a linked list containing the smallest value.
    
    Parameters
    ----------
    before: function of type ((a, a) -> bool)
        returns True if the first argument is "before" the second.
    
    l: linked list of elements of type a.
    
    Returns
    -------
    linked list
        the position of the minimum value in the list.
    '''
    minimum = l
    while l is not None:
        if before(l, minimum):
            minimum = l
        l = l.next
    return minimum

def maximum(before, l):
    return minimum(lambda a, b: not before(a, b), l)

In [ ]:
def by(key):
    return lambda a, b: eval(f'a.{key}') < eval(f'b.{key}')

In [ ]:
print(maximum(by('uid'), directory))
print(maximum(by('lastname'), directory))

In [ ]:
def swap_contents(this, that):
    '''
    Swap the contents of two directory entries.
    
    Parameters
    ----------
    one: Directory
    
    other: Directory
    
    Notes
    -----
    In case one of the two is None, does not swap.
    
    '''
    if this is not None and that is not None:
        (this.uid, this.zipcode, this.firstname, this.lastname,
         that.uid, that.zipcode, that.firstname, that.lastname) = (
         that.uid, that.zipcode, that.firstname, that.lastname, 
         this.uid, this.zipcode, this.firstname, this.lastname)

def inplace_selection_sort(by_what, l):
    '''
    Sort a Directory by selection sort.
    
    Parameters
    ----------
    by_what: function of type ((a, a) -> bool)
        return True if the first argument is "before" the second.
    
    l: Directory.
    '''
    while l is not None:
        current_minimum = minimum(by_what, l)
        swap_contents(l, current_minimum)
        l = l.next

def copied(l):
    '''
    Construct a list with the same content as the input list.
    
    Parameters
    ----------
    l: Directory.
    '''
    head = Directory(None, None, None, None)
    result = head  
    while l is not None:
        result.next = Directory(l.uid, l.zipcode, l.firstname, l.lastname)
        l, result = l.next, result.next
    return head.next

In [ ]:
dir_by_uid = copied(directory)
print(dir_by_uid)
inplace_selection_sort(by('uid'), dir_by_uid)
print(dir_by_uid)

In [ ]:
dir_by_firstname = copied(directory)
print(dir_by_firstname)
inplace_selection_sort(by('firstname'), dir_by_firstname)
print(dir_by_firstname)

In [ ]:
def selection_sorted(compare, l):
    '''
    Create a sorted Dictionary by selection sort.
    
    Parameters
    ----------
    compare: function of type ((a, a) -> bool)
        return True if the first argument is "before" the second.
    
    l: Directory.
    '''
    copy = copied(l)
    inplace_selection_sort(compare, copy)
    return copy

In [ ]:
print('input', directory)
print('output', selection_sorted(by('uid'), directory))

In [ ]:
from string import ascii_uppercase, ascii_lowercase

def as_directory(container):
    '''
    Create a Directory filled with random information and specified uids from a given container.
    
    Parameters
    ----------
    container: any object supporting the 'for ... in ...' syntax.
    
    Returns
    -------
    Directory
        a 'Directory' with uids obtained from the container and random data.
    '''
    head = Directory(None, None, None, None)
    result = head
    for i in container:
        result.next = Directory(i,
            int(''.join(map(str, random.sample(range(10), k=4)))),
            ''.join(random.sample(ascii_uppercase, k=1) + random.sample(ascii_lowercase, k=3)),
            ''.join(random.sample(ascii_uppercase, k=1) + random.sample(ascii_lowercase, k=3)))
        result = result.next
    return head.next

In [ ]:
random_dir = as_directory(random.sample(range(10), k=5))
print(random_dir)

In [ ]:
print(random_dir,
      selection_sorted(by('uid'), random_dir),
      selection_sorted(by('uid'),
                     selection_sorted(by('uid'), random_dir)))

In [ ]:
def generic_filter(condition, l):
    '''
    Linked list of all entries that fulfill the condition.
    
    Parameters
    ----------
    condition: function of type (linked list -> bool).
    
    l: linked list to be filtered
    
    Returns
    -------
    linked list
        containing the elements for which the condition is True.
    
    Notes
    -----
    If no element fulfills the condition, the returned linked list will be empty.
    '''
    L = l.__class__
    number_of_args = len(L.__init__.__code__.co_varnames) - 1
    
    head = L(*([None] * number_of_args))
    result = head
    while l is not None:
        if condition(l):
            result.next = L(*map(eval, [f'l.{key}' for key in l.__dict__ if key != 'next']))
            result = result.next
        l = l.next
    return head.next

print(generic_filter(lambda x: x.data % 2 == 1, my_list))
print(generic_filter(lambda x: x.uid % 2 == 1, random_dir))